# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис, пользователям теперь доступно редактирование и дополнение описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Создадим инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 
Для этого обучим модель, котторая будет классифицировать комментарии на токсичные(1) и нейтральные(0).

In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import torch
import transformers
import ydata_profiling

from catboost import CatBoostClassifier
from tqdm import notebook

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

## Обработка данных

In [2]:
try:
    df = pd.read_csv('datasets/toxic_comments.csv')
except:
    df = pd.read_csv('/datasets/toxic_comments.csv')
df.shape

(159292, 3)

В нашем распоряжении очень большой датасет, для экономии времени ограничимся семплом в 500 строк.

In [3]:
try:
    df = pd.read_csv('datasets/toxic_comments.csv').sample(500)
except:
    df = pd.read_csv('/datasets/toxic_comments.csv').sample(500)

In [4]:
df.head()

,Unnamed: 0,text,toxic
129000,129132,I want to argue the fact that Kaká is a non-EU...,0
18667,18686,"""\n\n-""""Supernatural"""" has an incorrect pronou...",0
124057,124186,I'll message you on your talk page 'cause I ju...,0
52083,52140,I have been more than civil with the people wh...,0
94188,94280,How about because it's not in Wales? It's in E...,0


In [5]:
df.columns

Index(['Unnamed: 0', 'text', 'toxic'], dtype='object')

В датасете есть лишний столбец, дублирующий индексы, избавимся от него.

In [6]:
df = df.drop('Unnamed: 0', axis=1)

%%time
ydata_profiling.ProfileReport(df) 

Тексты комментариев в датасете уникальны, повторов нет.

Для создания модели возьмём веса у уже обученной модели toxic-bert и с их помощью полоучим эмбеддинги для комментариев из нашего датасета.

In [7]:
from transformers import AutoTokenizer, AutoModel

pretrained_weights = 'unitary/toxic-bert'
tokenizer = AutoTokenizer.from_pretrained(pretrained_weights)
model = AutoModel.from_pretrained(pretrained_weights)

Создадим последовательности токенов для наших комментариев:

In [8]:
%%time
tokenized = df['text'].apply(
    lambda x: tokenizer(x, padding=True, truncation=True))

CPU times: user 228 ms, sys: 0 ns, total: 228 ms
Wall time: 1.1 s


Проверим, что последовательности токенов не превышают 512:

In [9]:
max_len = 0
for i in tokenized.values:
    if len(i['input_ids']) > max_len:
        max_len = len(i['input_ids'])
max_len

512

Дополним последовательности токенов короче 512 нулями и создадим маску для этих нулей, чтобы модель не принимала их во внимание.

In [10]:
padded = np.array([i['input_ids'] + [0]*(max_len - len(i['input_ids'])) for i in tokenized.values])

In [11]:
attention_mask = np.where(padded != 0, 1, 0)

Получим эмбеддинги моделей:

In [12]:
%%time
batch_size = 10
embeddings = []
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model = model.to(device)

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])        
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/50 [00:00<?, ?it/s]

CPU times: user 32min 26s, sys: 5min 31s, total: 37min 58s
Wall time: 13min 42s


## Обучение моделей

### Модель логистической регрессии

Объединим полученные эмбеддинги в признаки и выделим целевой признак:

In [13]:
features = np.concatenate(embeddings)
target = df['toxic']

Разобъём выборки на обучающую и тестовую:

In [14]:
features_train, features_test, target_train, target_test = \
train_test_split(features, target, test_size=0.2, stratify=target)

Обучим модель логистической регрессии на обучающей выборке:

In [15]:
lg_params = {
    "C":np.logspace(-3,3,7), 
    }
lg_model = LogisticRegression(max_iter=500)

lg_grid = GridSearchCV(lg_model,
                       lg_params,
                       cv=5)

In [16]:
%%time
lg_grid.fit(features_train, target_train)

CPU times: user 4.86 s, sys: 4.95 s, total: 9.81 s
Wall time: 1.48 s


GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=500),
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])})

In [17]:
lg_grid.best_score_

0.99

### CatBoostClassifier

In [18]:
cat_model = CatBoostClassifier(loss_function='Logloss', silent=True)

In [19]:
cat_params = {
    'iterations': [50, 100],
    'learning_rate': np.logspace(-3, -1, 5),
    'depth': [d for d in range(2, 11)],
    'l2_leaf_reg': np.logspace(-1, 1, 3)
             }

cat_grid = RandomizedSearchCV(cat_model, cat_params, n_iter=2)

In [20]:
%%time
cat_grid.fit(features_train, target_train)

CPU times: user 13min 48s, sys: 8.92 s, total: 13min 57s
Wall time: 2min 24s


RandomizedSearchCV(estimator=<catboost.core.CatBoostClassifier object at 0x7fafc1c3a430>,
                   n_iter=2,
                   param_distributions={'depth': [2, 3, 4, 5, 6, 7, 8, 9, 10],
                                        'iterations': [50, 100],
                                        'l2_leaf_reg': array([ 0.1,  1. , 10. ]),
                                        'learning_rate': array([0.001     , 0.00316228, 0.01      , 0.03162278, 0.1       ])})

In [21]:
cat_grid.best_score_

0.9875

### Light GBM Classifier

In [22]:
lgbm_model = lgb.LGBMRegressor(verbose=-1)

lgbm_params = {
    'num_leaves': [31, 41, 51],
    'min_child_samples': [5, 15],
    'max_depth': [10, 20],
    'learning_rate': np.logspace(-3, -1, 5),
    'reg_alpha': np.logspace(-4, -2, 2)
                                           }

lgbm_grid = RandomizedSearchCV(lgbm_model, lgbm_params, n_iter=2, verbose=False)

In [23]:
%%time
lgbm_grid.fit(features_train, target_train)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
CPU times: user 33.1 s, sys: 141 ms, total: 33.2 s
Wall time: 11 s


RandomizedSearchCV(estimator=LGBMRegressor(verbose=-1), n_iter=2,
                   param_distributions={'learning_rate': array([0.001     , 0.00316228, 0.01      , 0.03162278, 0.1       ]),
                                        'max_depth': [10, 20],
                                        'min_child_samples': [5, 15],
                                        'num_leaves': [31, 41, 51],
                                        'reg_alpha': array([0.0001, 0.01  ])},
                   verbose=False)

In [24]:
lgbm_grid.best_score_

0.8706793278659453

In [25]:
grids = [lg_grid, lgbm_grid, cat_grid]

models_data = []

for grid in grids:
    # id модели с лучшими параметрами
    best_id = np.argmin(grid.cv_results_['rank_test_score'])
    models_data.append(
        [
            # лучшее значение метрики на обучающей выборке
            (grid.cv_results_['mean_test_score'][best_id]).round(3),
            # время обучения модели
            grid.cv_results_['mean_fit_time'][best_id].round(3),
            # время плучения предсказаний
            grid.cv_results_['mean_score_time'][best_id].round(3)
        ]
    ) 
models_df = pd.DataFrame(data=models_data,
                         index=['LinearRegressor', 'LightGBM', 'CatBoost'],
                         columns=['лучшая метрика на обучающей выборке', 'время обучения модели', 'время получения предсказаний']
                        )
display(models_df)

,лучшая метрика на обучающей выборке,время обучения модели,время получения предсказаний
LinearRegressor,0.990,0.025,0.002
LightGBM,0.871,1.340,0.002
CatBoost,0.988,19.406,0.003


Модель LightGBM показывает результаты ниже, чем остальные модели, а модель CatBoost очень долго обучается, поэтому для получения метрики на тестовой выборке будем использовать модель логистической регресии.

In [26]:
target_predicted = lg_grid.best_estimator_.predict(features_test)

f1_score(target_test, target_predicted)

0.9166666666666666

Значение метрики на целевом уровне.

## Выводы по проделанной работе:
1. Данные были получены в хорошем качестве и практически не требовали предобработки.
1. Модель была обучена на семпле из 500 случайных комментариев.
1. В качестве основы модели для обучения была выбрана готовая модель 'toxic-bert' и с помощью неё были получены эмбеддинги, которые были использованы в качестве признаков для моделей классификации.
1. Были обучены несколько моделей с различными гиперпараметрами, самый лучший результат показала модель логистической регрессии.
1. Значение метрики f1 на тестовой выборке достигло целевого уровня.